In [1]:
import os, sys
project_root = os.path.abspath("../..")       # ajusta según dónde esté el notebook
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Importa tu módulo compilado
from cython_dbscan.dbscan_core import dbscan

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.Segmenter_v2.Segmenter import build_matrix_from_res8
import networkx as nx
import h3

df = pd.read_parquet('../../Data/sample_jan_2021.parquet')
df.head()

,mmsi,longitude,latitude,imo,vessel_name,callsign,vessel_type,vessel_type_cargo,length,width,destination,draught,sog,cog,heading,nav_status,dt_pos_utc,vessel_type_main,H3_int_index_15,res_8
1172258,257600000.0,10.810000,55.746667,9854478,SYDSTRAUM,LAGH7,Tanker,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",129.0,20.0,FI HMN > SE MMA,8.1,13.0,7.0,154.0,Under Way Using Engine,2021-01-01 00:00:50,None,644562452806690699,881f222c31fffff
1172259,257600000.0,10.810388,55.748605,9854478,SYDSTRAUM,LAGH7,Tanker,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",129.0,20.0,FI HMN > SE MMA,8.1,13.4,6.6,8.0,Under Way Using Engine,2021-01-01 00:01:00,None,644562452806187528,881f222c31fffff
1172260,259372000.0,24.117868,57.007650,9118496,CHRISTINA,LIJG3,Tanker,No Additional Information,124.0,20.0,LV RIX,5.6,0.0,316.9,47.0,Moored,2021-01-01 00:01:05,Oil And Chemical Tanker,644566704828286749,881f600c35fffff
1172261,257550000.0,0.562263,54.127452,9854466,SALTSTRAUM,LAJU6,Tanker,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",129.0,20.0,ANTWERP,6.0,13.7,136.3,138.0,Under Way Using Engine,2021-01-01 00:01:14,None,644459251375659225,88194464e7fffff
1172262,257550000.0,0.561667,54.126667,9854466,SALTSTRAUM,LAJU6,Tanker,"Carrying DG,HS or MP,IMO hazard or Pollutant C...",129.0,20.0,ANTWERP,6.0,13.0,136.0,0.0,Under Way Using Engine,2021-01-01 00:01:17,None,644459251374645290,88194464e7fffff


In [ ]:
#TODO agrupar cada res_8 con sus res_15, agupar los res_8 en su res_7 correspondiente


#calcular densidad por celda res_8
res8_counts = df.groupby('res_8').size().reset_index(name='count')

In [10]:
G = nx.Graph()
# Añadir nodos al grafo
for idx, row in res8_counts.iterrows():
    hex_id = row['res_8']
    count = row['count']
    G.add_node(hex_id, density=count)
    # Añadir vecinos del hexágono (1 anillo)
    for neighbor in h3.grid_disk(hex_id, 1):
        if neighbor != hex_id and neighbor in res8_counts["res_8"].values:
            G.add_edge(hex_id, neighbor)

KeyboardInterrupt: 

In [ ]:
# Mostrar nodos más densos
top_5 = sorted(G.nodes(data=True), key=lambda x: x[1]["density"], reverse=True)[:5]
for h, d in top_5:
    print(f"H3 res_8: {h}, Puntos: {d['density']}")

H3 res_8: 881f25d939fffff, Puntos: 1903
H3 res_8: 8808329509fffff, Puntos: 1351
H3 res_8: 881f2d14d7fffff, Puntos: 1020
H3 res_8: 88196ba157fffff, Puntos: 941
H3 res_8: 881f25d943fffff, Puntos: 857
